# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [25]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [27]:
# Drop unnecessary columns
new_df = df.drop(columns='timestamp', axis=1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [29]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(new_df[['userId', 'movieId', 'rating']], reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [31]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [32]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [33]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
param_grid = {'n_factors': [50, 100], 'n_epochs': [20, 30], 'lr_all': [0.005, 0.01], 'reg_all': [0.02, 0.1]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_score['mae'])

0.862843050935557
0.6618860179205889


In [35]:
# print out optimal parameters for SVD after GridSearch
print(gs.best_params['rmse'])

{'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.1}


In [36]:
# cross validating with KNNBasic
knn_basic = KNNBasic()

cross_validate(knn_basic, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9446  0.9538  0.9364  0.9542  0.9504  0.9479  0.0067  
MAE (testset)     0.7202  0.7331  0.7226  0.7295  0.7277  0.7266  0.0047  
Fit time          0.19    0.24    0.21    0.22    0.27    0.23    0.03    
Test time         1.69    2.05    1.92    2.53    2.03    2.04    0.27    


{'test_rmse': array([0.94462986, 0.95383924, 0.93644472, 0.9541696 , 0.95042816]),
 'test_mae': array([0.72021227, 0.73312236, 0.72259532, 0.72954958, 0.72768956]),
 'fit_time': (0.19096636772155762,
  0.24460649490356445,
  0.21311616897583008,
  0.21645498275756836,
  0.2695910930633545),
 'test_time': (1.6886563301086426,
  2.0537140369415283,
  1.920938491821289,
  2.52518367767334,
  2.0336673259735107)}

In [37]:
# print out the average RMSE score for the test set
test_rmse_scores = np.array([0.97646619, 0.97270627, 0.97874535, 0.97029184, 0.96776748])
average_rmse = np.mean(test_rmse_scores)
average_rmse

0.973195426

In [38]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline()

cross_validate(knn_baseline, data, measures=['RMSE'], cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8758  0.8746  0.8757  0.8796  0.8677  0.8747  0.0039  
Fit time          0.56    0.48    0.41    0.40    0.47    0.46    0.06    
Test time         2.53    2.79    2.45    2.84    2.52    2.63    0.16    


{'test_rmse': array([0.8757563 , 0.87464918, 0.87565637, 0.87955064, 0.86766944]),
 'fit_time': (0.5568163394927979,
  0.47776293754577637,
  0.413942813873291,
  0.39937806129455566,
  0.4729745388031006),
 'test_time': (2.5335607528686523,
  2.785728693008423,
  2.45343279838562,
  2.8428421020507812,
  2.5187392234802246)}

In [39]:
# print out the average score for the test set
test_rmse_scores = np.array([0.87268017, 0.88765352, 0.87311917, 0.88706914, 0.87043399])
average_rmse = np.mean(test_rmse_scores)
print(average_rmse)

0.878191198


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [40]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [41]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [42]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [43]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0800692338488793, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [44]:
def movie_rater(movie_df,num, genre=None):
    rating_list = []
    seen_movies = set() 

    for _ in range(num):
        if genre:
            genre_movies = movie_df[movie_df['genres'].str.contains(genre)]
            if genre_movies.empty:
                print(f"No movies found for genre '{genre}'. Skipping this iteration.")
                continue
            movie = genre_movies.sample(1)
        else:
            movie = movie_df.sample(1)

        movie_id = movie['movieId'].iloc[0]
        movie_title = movie['title'].iloc[0]
        
        if movie_id in seen_movies:
            continue
        seen_movies.add(movie_id)

        print(f"Movie Title: {movie_title}")

        while True:
            rating = input("Rate the movie (0.5-5, or 's' to skip): ")
            if rating.lower() == 's':
                break
            try:
                rating = float(rating)
                if 0.5 <= rating <= 5:
                    rating_list.append({'userId': 1, 'movieId': movie_id, 'rating': rating})
                    break
                else:
                    print("Invalid rating. Please enter a value between 0.5 and 5.")
            except ValueError:
                print("Invalid input. Please enter a number between 0.5 and 5, or 's' to skip.")
    return rating_list     

In [ ]:
# try out the new function here!
new_ratings = movie_rater(df_movies, 5, genre='Comedy')
new_ratings

Movie Title: Wackness, The (2008)


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [ ]:
user_rating

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [ ]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = pd.DataFrame(new_ratings)
combined_df = pd.concat([new_df, new_ratings_df], ignore_index=True)

In [ ]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(combined_ratings[['userId', 'movieId', 'rating']], reader)
dataset = data.build_full_trainset()

In [ ]:
# train a model using the new combined DataFrame
svd = SVD(n_factors=100, n_epochs=30, lr_all=0.01, reg_all=0.1)
svd.fit(dataset)    

In [ ]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)   
predictions = []
for movie_id in df_movies['movieId'].unique():
    if movie_id not in combined_ratings[combined_ratings['userId'] == 1]['movieId'].values:
        prediction = svd.predict(1, movie_id)
        predictions.append((movie_id, prediction.est))


top_n_recommendations = ranked_movies[:n]
recommendations = []
for movie_id, predicted_score in ranked_movies[:n]:
    movie_title = df_movies[df_movies['movieId'] == movie_id]['title'].iloc[0]
    recommendations.append((movie_title, predicted_score))
print(recommendations)

In [ ]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(predictions, key=lambda x: x[1], reverse=True)

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [ ]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
    for i in range(n):
    movie_id = user_ratings[i][1]
    recommendation = movie_title_df[movie_title_df['movieId'] == movie_id]['title']
    print(f"Recommendation # {i+1} : {recommendation.iloc[0]}")
            
recommended_movies(ranked_movies,df_movies,5)

## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.